RAG Pipeline-Data Ingestion to Vector DB

In [2]:
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
loader=DirectoryLoader("data/pdf_files", glob="*.pdf",loader_cls=PyMuPDFLoader)
loader.load()

e:\RAG\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'producer': '', 'creator': 'Microsoft Word', 'creationdate': '2025-09-18T20:40:19-07:00', 'source': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'file_path': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'total_pages': 9, 'format': 'PDF 1.7', 'title': '', 'author': 'python-docx', 'subject': '', 'keywords': '', 'moddate': '2025-09-18T20:40:19-07:00', 'trapped': '', 'modDate': "D:20250918204019-07'00'", 'creationDate': "D:20250918204019-07'00'", 'page': 0}, page_content='Civil Supplies Project Documentation \nThe Civil Supplies Project automates the monitoring and management of warehouse activities using \nthree integrated systems: \n- Facial Recognition System for identifying individuals. \n- Gunny Bags Counting System for tracking loading and unloading activities. \n- Number Plate Recognition System for vehicle access control. \nThese systems work together through a unified pipeline that processes live video feeds, enhancing \noperational effici

In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from pathlib import Path
# Try this first (newer LangChain versions)
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # Fallback for older versions
    from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def read_pdf_files(pdf_dir):
    all_documents=[]
    pdf_dir=Path(pdf_dir)
    pdf_files=list(pdf_dir.glob("*.pdf"))
    print(len(pdf_files), " pdf file/s founded")
    for file in pdf_files:
        print("file name:",file.name)
        loader=PyMuPDFLoader(str(file))
        documents=loader.load()
        for doc in documents:
            doc.metadata['source_file']=file.name
            doc.metadata['file_type']="pdf"
        all_documents.extend(documents)
        print(len(all_documents))
    return all_documents
all_pdf_documents=read_pdf_files("data/pdf_files")
all_pdf_documents




1  pdf file/s founded
file name: Civil-Supplies-Documentation-V1.pdf
9


[Document(metadata={'producer': '', 'creator': 'Microsoft Word', 'creationdate': '2025-09-18T20:40:19-07:00', 'source': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'file_path': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'total_pages': 9, 'format': 'PDF 1.7', 'title': '', 'author': 'python-docx', 'subject': '', 'keywords': '', 'moddate': '2025-09-18T20:40:19-07:00', 'trapped': '', 'modDate': "D:20250918204019-07'00'", 'creationDate': "D:20250918204019-07'00'", 'page': 0, 'source_file': 'Civil-Supplies-Documentation-V1.pdf', 'file_type': 'pdf'}, page_content='Civil Supplies Project Documentation \nThe Civil Supplies Project automates the monitoring and management of warehouse activities using \nthree integrated systems: \n- Facial Recognition System for identifying individuals. \n- Gunny Bags Counting System for tracking loading and unloading activities. \n- Number Plate Recognition System for vehicle access control. \nThese systems work together through a unifie

In [5]:
def split_document(documents,chunk_size=1000,chunk_overlap=100):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                 chunk_overlap=chunk_overlap, 
                                                 length_function=len,
                                                 separators=["\n\n", "\n", " ", ""])
    split_docs=text_splitter.split_documents(documents)
    print(f"splitted {len(all_pdf_documents)} into {len(split_docs)} chunks")
    print(split_docs)
    return split_docs
chunks=split_document(all_pdf_documents)


splitted 9 into 14 chunks
[Document(metadata={'producer': '', 'creator': 'Microsoft Word', 'creationdate': '2025-09-18T20:40:19-07:00', 'source': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'file_path': 'data\\pdf_files\\Civil-Supplies-Documentation-V1.pdf', 'total_pages': 9, 'format': 'PDF 1.7', 'title': '', 'author': 'python-docx', 'subject': '', 'keywords': '', 'moddate': '2025-09-18T20:40:19-07:00', 'trapped': '', 'modDate': "D:20250918204019-07'00'", 'creationDate': "D:20250918204019-07'00'", 'page': 0, 'source_file': 'Civil-Supplies-Documentation-V1.pdf', 'file_type': 'pdf'}, page_content='Civil Supplies Project Documentation \nThe Civil Supplies Project automates the monitoring and management of warehouse activities using \nthree integrated systems: \n- Facial Recognition System for identifying individuals. \n- Gunny Bags Counting System for tracking loading and unloading activities. \n- Number Plate Recognition System for vehicle access control. \nThese systems work

Embeddings generator

In [6]:
import numpy as np 
from sentence_transformers import SentenceTransformer 
import chromadb 
from chromadb.config import Settings 
import uuid 
from typing import List, Dict, Any, Tuple 
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
     """Handles document embedding generation using SentenceTransformer""" 
     def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
         #Initialize the embedding manager Args: model name: HuggingFace model name for sentence embeddings 
         self.model_name= model_name 
         self.model=None 
         self._load_model()
     def _load_model(self): 
        """Load the SentenceTransformer model""" 
        try: 
            print(f"Loading embedding model: {self.model_name}") 
            self.model=SentenceTransformer(self.model_name) 
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}") 
        except Exception as e: 
            print(f"Error loading model {self.model_name}: {e}") 
     def generate_embeddings(self,texts:List[str]) :
         if not self.model:
             print("model not loaded")
         embeddings=self.model.encode(texts,show_progress_bar=True)
         print(f"length of embeddings are {len(embeddings)}")
         return embeddings
embedding_manager=EmbeddingManager()

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


Vector store

In [8]:
class VectorStore:
    def __init__(self,collection_name:str="pdf_documents",persistent_directory:str="vector_store"):
        self.collection_name=collection_name
        self.persistent_directory=persistent_directory
        self.collection=None
        self.client=None
        self._initialize_store()
    def _initialize_store(self):
        os.makedirs(self.persistent_directory,exist_ok=True)
        self.client=chromadb.PersistentClient(path=self.persistent_directory)
        self.collection=self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description":"Am creating the collection using ChromaDB"}
            
        )
        print(f"collection is created with name: {self.collection_name}")
        # print(f"number of documents in {self.collection_name}")
    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        if len(documents)!=len(embeddings):
            return None
        ids=[]
        metadatas=[]
        documents_text=[]
        embeddings_list=[]
        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate unique ID 
            doc_id=f"doc_(uuid.uuid4().hex[:8])_{i}" 
            ids.append(doc_id) 
            # Prepare metadata 
            metadata=dict(doc.metadata) 
            metadata['doc_index'] = i 
            metadata['content_length'] = len(doc.page_content) 
            metadatas.append(metadata) 
            #Document content 
            documents_text.append(doc.page_content) 
            # Embedding
            embeddings_list.append(embedding.tolist())
            try:
                self.collection.add(
                    ids=ids,
                    embeddings=embeddings_list,
                    metadatas=metadatas,
                    documents=documents_text
                )
                print(f"successfully added the document to the {self.collection_name}")
                print(f"Total documents in the colllection: {self.collection.count()}")
            except:
                print("Error adding documents to the store")
vector_store=VectorStore()

collection is created with name: pdf_documents


In [9]:
texts=[doc.page_content for doc in chunks]
print("Text chunks are; ",texts)
len(texts)

Text chunks are;  ['Civil Supplies Project Documentation \nThe Civil Supplies Project automates the monitoring and management of warehouse activities using \nthree integrated systems: \n- Facial Recognition System for identifying individuals. \n- Gunny Bags Counting System for tracking loading and unloading activities. \n- Number Plate Recognition System for vehicle access control. \nThese systems work together through a unified pipeline that processes live video feeds, enhancing \noperational efficiency, security. \n \nPrerequisites & Configuration \nBefore running the Civil Supplies Warehouse pipeline, ensure these secrets are configured in GitHub: \n Azure Configuration \n• \nAZURE_TENANT_ID – Tenant ID for Azure authentication. \n• \nAZURE_CLIENT_ID – Client ID of the registered Azure application. \n• \nAZURE_CLIENT_SECRET – Client secret for the registered Azure application. \n• \nAZURE_STORAGE_ACCOUNT_NAME – Storage account name. \n• \nAZURE_CONTAINER_NAME – Container name for st

14

In [10]:
embeddings=embedding_manager.generate_embeddings(texts)
print(embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]

length of embeddings are 14
[[-0.04572309  0.02351205 -0.07256225 ... -0.00854249 -0.0196133
  -0.10350119]
 [ 0.03053385 -0.03236363 -0.10852879 ...  0.01276315  0.04849378
  -0.05026677]
 [-0.01106906 -0.04497927 -0.09534009 ...  0.06563845 -0.00387122
  -0.05113655]
 ...
 [-0.07153378  0.02568216 -0.04570009 ...  0.00066344  0.01521763
  -0.07928558]
 [-0.04274938 -0.0575447  -0.03892815 ... -0.06579547  0.03259268
  -0.03001297]
 [-0.01270368  0.02244613  0.06478529 ... -0.04106241 -0.02070728
  -0.0289937 ]]


In [11]:
vector_store.add_documents(chunks,embeddings)

successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the colllection: 14
successfully added the document to the pdf_documents
Total documents in the coll

Creating RAG Retrieval

In [12]:
class RAGRetriever:
    def __init__(self,vectorstore:VectorStore,embedding_manager:EmbeddingManager):
        self.vectorstore=vectorstore
        self.embedding_manager=embedding_manager
    def retrieve(self,query:str,top_k:int=5,similarity_threshold:float=0.0):
        print(f"Retrieving documents for the query: {query}")
        print(f"default top_k is:{top_k} and score_threshold is {similarity_threshold}")
        query_embedding=self.embedding_manager.generate_embeddings([query])[0]
        # print(query_embedding)
        # print([query_embedding.tolist()])
        try:
            results=self.vectorstore.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            print(results)
            retrieved_docs=[]
            if results['documents'] and results['documents'][0]:
                print(f"Retrieving results for the query:{query}")
                documents=results['documents'][0]
                metadatas=results['metadatas'][0]
                distances=results['distances'][0]
                ids=results['ids'][0]
                for i,(document,metadata,id,distance) in enumerate(zip(documents,metadatas,ids,distances)):
                    cosine=1-distance
                    if cosine<=similarity_threshold:
                        retrieved_docs.append(
                            {
                                "id":id,
                                "content":document,
                                "metadata":metadata,
                                "similarity_score":cosine,
                                "distance":distance,
                                "rank":i+1
                            }
                        )
                print(f"Retrieved {len(retrieved_docs)} after filtering")
            else:
                print("No records found")
            return retrieved_docs
        except:
            print("failed retrieving the results")

rag_retriever=RAGRetriever(vector_store,embedding_manager)
retrieved_results=rag_retriever.retrieve("about civil supplies project")
# print(retrieved_results)
for i,result in enumerate(retrieved_results):
    print("------------------------------------------------------------------")
    print(f"Result: {i+1}: {result['content']}")

Retrieving documents for the query: about civil supplies project
default top_k is:5 and score_threshold is 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.04it/s]

length of embeddings are 1
{'ids': [['doc_(uuid.uuid4().hex[:8])_2', 'doc_(uuid.uuid4().hex[:8])_6', 'doc_(uuid.uuid4().hex[:8])_5', 'doc_(uuid.uuid4().hex[:8])_4', 'doc_(uuid.uuid4().hex[:8])_8']], 'embeddings': None, 'documents': [['Designed and deployed modules for a government surveillance product, including hands-on evaluations\nof CCTV camera placements in government warehouses and on-site demonstrations in government\noffices.\nCollaborated with stakeholders to ensure compliance with operational and security requirements,\nproviding actionable insights for real-world government environments.\nSRAVAN POTNURU\nP R O J E C T S\nProject: ParaBot (AI Chat + File Summarizer) | FastAPI, React, OCR, Gemini | GitHub  \nImplemented an AI assistant that processes text, images, PDFs, and website content with summarization and Q&A\ncapabilities.\nProject: Stress Prediction Based on Sleeping Patterns | Python, ML, Streamlit | GitHub\nDeveloped a machine learning model that predicts stress lev